Import Libraries

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from xgboost import XGBRegressor



In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import shap



Load Enhanced Dataset

In [16]:
df = pd.read_csv('/content/drive/MyDrive/ABB/data/enhanced_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ABB/data/enhanced_test.csv')

In [17]:
# Drop unnecessary columns
X = df.drop(columns=['Unnamed: 0', 'Item_Outlet_Sales', 'Log_Sales', 'Item_Identifier'])
y = df['Log_Sales']


In [18]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = []
val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"Fold {fold+1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMRegressor(
        objective='regression',
        n_estimators=1000,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.3,
        random_state=42,
         early_stopping_rounds=50,
        verbose=100
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
    )

    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(np.expm1(y_val), np.expm1(preds)))
    val_scores.append(rmse)
    oof_preds.extend(preds)

print(f"\nMean CV RMSE: {np.mean(val_scores):.4f}")


Streaming output truncated to the last 5000 lines.
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and depth = 6
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 15 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 27 and depth = 6
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and depth = 6
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 24 and depth = 6
[LightGBM] [Warning] No further splits with positive g

In [20]:
# Align test data
X_test = df_test[X.columns]
test_preds_log = model.predict(X_test)
test_preds = np.expm1(test_preds_log)

# Create submission
submission = df_test[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = test_preds

# Inverse transform (if label encoded)
with open('/content/drive/MyDrive/ABB/label_encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

for col in ['Item_Identifier', 'Outlet_Identifier']:
    if col in encoders:
        submission[col] = encoders[col].inverse_transform(df_test[col].astype(int))

submission.to_csv('/content/drive/MyDrive/ABB/lgbm_submission.csv', index=False)
print("Submission saved: lgbm_submission.csv")


Submission saved: lgbm_submission.csv
